# Preparing Data for Forecasting

The goal of this notebook is to create the necessary dataframes to do forecasting. This is my attempt at building something to analyze the statistics as time series. This is, perhaps a dumb approach. Here is the idea. I will create aux dataframes that contain the stats of each team. I will then build a dataframe where the contents is the average performance of the team over the last five games. I will make a couple passes at this. For a first pass, I may not include statistics about what they allowed their opponents to do. I am also not be concerned with whether or not they were the home team.

In [1]:
import os

home_dir = "/workspace/nba-study"
csv_dir = "datasets/clean/csv"
os.chdir(home_dir)
current_dir = os.getcwd()
print(current_dir)

/workspace/nba-study


Now we load the relevant data.

In [2]:
from src.utils import get_df

game_data = get_df("game_data_clean",csv_dir)

In [10]:
import pandas as pd
import numpy as np

We are going to collect all of the team ids and make a dataframe for each of them. We will, in general, need to sort these dataframes by date. This may involve making some custom method/function or it may involve creating a column of datetime objects.

We checked that the set of ids is the same whether we use home or away teams.

In [17]:
team_ids = game_data['team_id_home'].unique()

def get_home_games(data,team_id)-> pd.DataFrame:
    return data.query(f"team_id_home == {team_id}")


def get_away_games(data, team_id)-> pd.DataFrame:
    return data.query(f"team_id_away == {team_id}")


def get_all_games(data, team_id) -> pd.DataFrame:
    return get_home_games(data,team_id).append(get_away_games(data,team_id))
    


In [16]:
team_1 = team_ids[0]
home_games_team_1 = get_home_games(game_data,team_1)
home_games_team_1.head()

,season_id,team_id_home,team_name_home,wl_home,fgm_home,fga_home,fg3m_home,fg3a_home,ftm_home,fta_home,...,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,day,month,year
game_id,,,,,,,,,,,,,,,,,,,,,
28500005,21985,1610612737,Atlanta Hawks,L,41.0,92.0,0.0,3.0,9.0,18.0,...,40.0,21.0,11.0,7.0,17.0,19.0,100.0,25,10,1985
28500030,21985,1610612737,Atlanta Hawks,W,37.0,86.0,1.0,3.0,27.0,38.0,...,44.0,15.0,8.0,3.0,12.0,26.0,87.0,29,10,1985
28500054,21985,1610612737,Atlanta Hawks,W,44.0,86.0,0.0,0.0,26.0,37.0,...,48.0,22.0,7.0,2.0,16.0,33.0,113.0,2,11,1985
28500103,21985,1610612737,Atlanta Hawks,W,43.0,79.0,1.0,1.0,21.0,32.0,...,46.0,17.0,6.0,4.0,24.0,26.0,101.0,13,11,1985
28500117,21985,1610612737,Atlanta Hawks,W,44.0,86.0,0.0,1.0,34.0,42.0,...,47.0,27.0,10.0,1.0,24.0,29.0,118.0,15,11,1985


In [23]:
all = get_all_games(game_data,team_1)
home = get_home_games(game_data,team_1)
away = get_away_games(game_data, team_1)
print(home.shape,away.shape,all.shape)


(1574, 40) (1577, 40) (3151, 40)


/tmp/ipykernel_3686/2048089440.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return get_home_games(data,team_id).append(get_away_games(data,team_id))


0

In [ ]:
def add_date(data:pd.DataFrame) -> pd.DataFrame:
    cols = set([col.lower() for col in data.columns])
    if not col.issubset(set('day', 'month', 'year')):
        raise ValueError("The requisite data is not present")
    return